<a href="https://colab.research.google.com/github/bstee615/usgs-machine-to-machine-API/blob/master/webtile_mgrs_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example code converting a web tile to MGRS tiles @10km precision

Used pip packages [mgrs](https://github.com/hobu/mgrs) to convert lat/long ↔ MGRS and [mercantile](https://mercantile.readthedocs.io/en/latest/index.html) to convert web tile ↔ lat/long.
The math is straightforward, see docs at [OpenStreetMap wiki](https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Lon..2Flat._to_tile_numbers_2) and [MGRS wiki](https://en.wikipedia.org/wiki/Military_Grid_Reference_System).

In [14]:
!pip install mgrs mercantile

First, get the MGRS coordinates that denote the Southwest and Northeast corner of the web tile.

In [15]:
# Go from webtile -> lat/long -> MGRS
import mercantile
import mgrs

mgrs = mgrs.MGRS()
mgrs_precision = 1  # 10km x 10km precision


def latlong_to_mgrs(lat, lon):
    return mgrs.toMGRS(lat, lon, MGRSPrecision=mgrs_precision)


def webtile_to_latlon(x, y, z):
    tile = mercantile.tile(x, y, z)
    bounds = mercantile.bounds(tile)
    return bounds.south, bounds.west, bounds.north, bounds.east


x, y, z = 21.5, 41, 9
print('Input web tile (xyz):', x, y, z)
lat_s, lon_w, lat_n, lon_e = webtile_to_latlon(x, y, z)
print('Extents (SWNE):', lat_s, lon_w, lat_n, lon_e)
sw = latlong_to_mgrs(lat_s, lon_w)
ne = latlong_to_mgrs(lat_n, lon_e)
print('Extents in MGRS (SW, NE):', sw, ne)

Input web tile (xyz): 21.5 41 9
Extents (SWNE): 40.97989806962013 21.09375 41.50857729743934 21.796875
Extents in MGRS (SW, NE): 34TEL03 34TEL69


There are actually more tiles on the interior of the web tile which should be filled in.

In [16]:
def collect_interiors(sw, ne):
    """
    Collect tiles on the interior of the NW and SE corners.
    Handles only the simple case where the tile is the same on the 100km square.
    """
    
    stub = sw[:-2]
    ne_stub = ne[:-2]
    assert stub == ne_stub, 'Tiles do not land on the same 100km square'

    w = int(sw[-2])
    s = int(sw[-1])
    e = int(ne[-2])
    n = int(ne[-1])
    assert w < e, 'Cannot iterate easting'
    assert s < n, 'Cannot iterate northing'

    tiles = []
    for easting in range(w, e):
        for northing in range(s, n+1):
            tile = stub + str(easting) + str(northing)
            tiles.append(tile)
    return tiles


tiles = collect_interiors(sw, ne)
print('All MGRS tiles in web tile extent:', ' '.join(tiles))

All MGRS tiles in web tile extent: 34TEL03 34TEL04 34TEL05 34TEL06 34TEL07 34TEL08 34TEL09 34TEL13 34TEL14 34TEL15 34TEL16 34TEL17 34TEL18 34TEL19 34TEL23 34TEL24 34TEL25 34TEL26 34TEL27 34TEL28 34TEL29 34TEL33 34TEL34 34TEL35 34TEL36 34TEL37 34TEL38 34TEL39 34TEL43 34TEL44 34TEL45 34TEL46 34TEL47 34TEL48 34TEL49 34TEL53 34TEL54 34TEL55 34TEL56 34TEL57 34TEL58 34TEL59


In [17]:
def intersects(tile, polygon):
    """Fake implementation! In reality this would be a spatial join (https://geopandas.org/docs/user_guide/mergingdata.html#spatial-joins)."""
    if tile in ('34TEL08', '34TEL07', '34TEL18'):
        return True
    else:
        return False


def filter_to_polygon(tiles, polygon):
    filtered_tiles = []
    for tile in tiles:
        if intersects(tile, polygon):
            filtered_tiles.append(tile)
    return filtered_tiles


polygon = [(21.08782035624389,41.4095885126781), (21.06876059500927,41.34439217626883), (21.1694810845141,41.41875148857508), (21.08782035624389,41.4095885126781)]  # Triangular polygon in linear ring form
filtered_tiles = filter_to_polygon(tiles, polygon)

print('MGRS tiles filtered to the requested polygons:', ' '.join(filtered_tiles))
print('The satellite images for these MGRS tiles would be downloaded from the server, clipped, and merged into a bitmap to return from the tile server.')

MGRS tiles filtered to the requested polygons: 34TEL07 34TEL08 34TEL18
The satellite images for these MGRS tiles would be downloaded from the server, clipped, and merged into a bitmap to return from the tile server.
